In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer,LlamaTokenizer, LlamaForCausalLM
import torch
import numpy as np
import torch.nn.functional as F
from einops import rearrange

/n/holylfs06/LABS/kempner_undergrads/Users/jujipotle/envs/iti/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
llama2_chat_70B_model_name = 'meta-llama/Meta-Llama-3-8B-Instruct'
llama2_chat_70B_model = AutoModelForCausalLM.from_pretrained(llama2_chat_70B_model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:  50%|█████     | 2/4 [00:09<00:09,  4.58s/it]


KeyboardInterrupt: 

In [10]:

def view_model_architecture(model):
    print(model)
    for i, layer in enumerate(model.model.layers):
        print(f"Layer {i}: {layer.self_attn.o_proj.bias}")

In [11]:
def edit_model_biases(model):
    for i, layer in enumerate(model.model.layers):
        num_heads = model.config.num_attention_heads
        hidden_size = model.config.hidden_size
        displacement = np.zeros((int(num_heads), int(hidden_size / num_heads)))
        displacement = torch.tensor(rearrange(displacement, 'h d -> (h d)'), device=layer.self_attn.o_proj.weight.device)
        layer.self_attn.o_proj.bias = torch.nn.parameter.Parameter(displacement.to(torch.float32))

In [12]:
def save_model(model, path):
    # model.config.oproj_bias = True
    model.config.attention_bias = True
    model.save_pretrained(path, safe_serialization=False, max_shard_size="10GB")

In [ ]:
huggy_edited_path = 'results_dump/huggy_edited'
view_model_architecture(huggy_auto_model)
edit_model_biases(huggy_auto_model)
view_model_architecture(huggy_auto_model)
save_model(huggy_auto_model, huggy_edited_path)
